In [ ]:
# ---------------------------------------------------------
# Imports
# ---------------------------------------------------------
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix

import mlflow
import mlflow.sklearn
import dagshub


!pip install -U "mlflow<3" "pydantic>=2"

# ---------------------------------------------------------
# Fix MLflow serialization issue: define named function
# ---------------------------------------------------------
def log1p_transform(x):
    return np.log1p(x)


# ---------------------------------------------------------
# MLflow + DagsHub Setup
# ---------------------------------------------------------
dagshub.init(repo_owner="menna1996", repo_name="avoidable_ed_ml_project", mlflow=True)
mlflow.set_experiment("Experiment_1_LogReg")


# ---------------------------------------------------------
# Load cleaned modeling dataset
# ---------------------------------------------------------
df = pd.read_parquet("modeling_dataset_clean.parquet")
X = df.drop(columns=["Avoidable_ED_Visit"])
y = df["Avoidable_ED_Visit"]


# ---------------------------------------------------------
# Train/Test Split
# ---------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


# ---------------------------------------------------------
# Identify column types
# ---------------------------------------------------------
numeric_cols = ["AGE_AT_END_REF_YR", "total_paid_amt"]
categorical_cols = ["SEX_IDENT_CD", "BENE_RACE_CD", "YEAR"]
body_system_cols = [col for col in X.columns if col.startswith("BodySystem_")]


# ---------------------------------------------------------
# Preprocessing Pipeline
# ---------------------------------------------------------
log_transformer = FunctionTransformer(log1p_transform)

preprocessor = ColumnTransformer(
    transformers=[
        ("num_standard", StandardScaler(), ["AGE_AT_END_REF_YR"]),
        ("num_log_minmax", Pipeline([
            ("log", log_transformer),
            ("minmax", MinMaxScaler())
        ]), ["total_paid_amt"]),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("body", StandardScaler(), body_system_cols)
    ]
)


# ---------------------------------------------------------
# Full Pipeline with Logistic Regression
# ---------------------------------------------------------
pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("clf", LogisticRegression(max_iter=1000))
])


# ---------------------------------------------------------
# Hyperparameter Tuning
# ---------------------------------------------------------
param_grid = {
    "clf__C": [0.01, 0.1, 1, 10],
    "clf__penalty": ["l2"],
    "clf__solver": ["lbfgs"]
}

grid = GridSearchCV(
    pipeline,
    param_grid,
    cv=3,
    scoring="f1",
    n_jobs=-1
)


# ---------------------------------------------------------
# Run MLflow Experiment
# ---------------------------------------------------------
with mlflow.start_run():

    grid.fit(X_train, y_train)
    mlflow.log_params(grid.best_params_)

    # Cross-validation
    cv3 = cross_validate(grid.best_estimator_, X_train, y_train, cv=3, scoring="f1")
    cv10 = cross_validate(grid.best_estimator_, X_train, y_train, cv=10, scoring="f1")

    mlflow.log_metric("cv3_f1_mean", cv3["test_score"].mean())
    mlflow.log_metric("cv3_f1_std", cv3["test_score"].std())
    mlflow.log_metric("cv10_f1_mean", cv10["test_score"].mean())
    mlflow.log_metric("cv10_f1_std", cv10["test_score"].std())

    # Train set evaluation
    y_pred_train = grid.predict(X_train)
    f1_train = f1_score(y_train, y_pred_train)
    tn, fp, fn, tp = confusion_matrix(y_train, y_pred_train).ravel()

    mlflow.log_metric("train_f1", f1_train)
    mlflow.log_metric("train_TP", tp)
    mlflow.log_metric("train_TN", tn)
    mlflow.log_metric("train_FP", fp)
    mlflow.log_metric("train_FN", fn)

    # Test set evaluation
    y_pred_test = grid.predict(X_test)
    f1_test = f1_score(y_test, y_pred_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()

    mlflow.log_metric("test_f1", f1_test)
    mlflow.log_metric("test_TP", tp)
    mlflow.log_metric("test_TN", tn)
    mlflow.log_metric("test_FP", fp)
    mlflow.log_metric("test_FN", fn)

    # Log model
    mlflow.sklearn.log_model(grid.best_estimator_, "logreg_model")


print("✅ Experiment #1 complete")
print("Best Params:", grid.best_params_)
print("Train F1:", f1_train)
print("Test F1:", f1_test)

MLFLOW.END_RUN(STATUS="FINISHED")

  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
Using cached pydantic-2.12.5-py3-none-any.whl (463 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 19.2 MB/s eta 0:00:00
Using cached typing_inspection-0.4.2-py3-none-any.whl (14 kB)

  Attempting uninstall: pydantic-core

    Found existing installation: pydantic_core 2.27.1

    Uninstalling pydantic_core-2.27.1:

      Successfully uninstalled pydantic_core-2.27.1

  Attempting uninstall: pydantic

    Found existing installation: pydantic 2.10.3

    Uninstalling pydantic-2.10.3:

      Successfully uninstalled pydantic-2.10.3

   -------------------------- ------------- 2/3 [pydantic]
   -------------------------- ------------- 2/3 [pydantic]
   -------------------------- ------------- 2/3 [pydantic]
   ---------------------------------------- 3/3 [pydantic]



Initialized MLflow to track repo "menna1996/avoidable_ed_ml_project"

Repository menna1996/avoidable_ed_ml_project initialized!

2025/12/13 23:29:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run marvelous-ant-601 at: https://dagshub.com/menna1996/avoidable_ed_ml_project.mlflow/#/experiments/0/runs/bf0801975d5642bf9a938c3b75f341e1
🧪 View experiment at: https://dagshub.com/menna1996/avoidable_ed_ml_project.mlflow/#/experiments/0


ImportError: cannot import name 'IS_PYDANTIC_V2_OR_NEWER' from 'mlflow.utils' (c:\Users\bruck\anaconda3\Lib\site-packages\mlflow\utils\__init__.py)